In [190]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
conn = pymongo.MongoClient()['steam']

In [3]:
cursor = conn['test'].find({})

In [4]:
df = pd.DataFrame(list(cursor))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 7 columns):
_id                656 non-null object
ban_status         656 non-null object
friends            656 non-null object
game_info          656 non-null object
profile_summary    656 non-null object
steamid            656 non-null object
user_info          656 non-null object
dtypes: object(7)
memory usage: 35.9+ KB


In [240]:
del df["_id"]

In [261]:
df["num_friends"] = map(lambda x: len(x), df["friends"])

In [267]:
df.columns

Index([                 u'friends',                u'game_info',
                u'profile_summary',                  u'steamid',
                     u'avatarfull',        u'commentpermission',
       u'communityvisibilitystate',            u'gameextrainfo',
                         u'gameid',             u'gameserverip',
              u'gameserversteamid',               u'lastlogoff',
                      u'loccityid',           u'loccountrycode',
                   u'locstatecode',              u'personaname',
                   u'personastate',            u'primaryclanid',
                   u'profilestate',                 u'realname',
                    u'timecreated',          u'CommunityBanned',
               u'DaysSinceLastBan',               u'EconomyBan',
               u'NumberOfGameBans',          u'NumberOfVACBans',
                      u'VACBanned',              u'num_friends'],
      dtype='object')

In [268]:
df.personastate.value_counts()

0    540
4     51
1     35
3     30
Name: personastate, dtype: int64

In [7]:
games = df.copy()
games = games[["steamid", "game_info"]]

In [8]:
#df['user_info'][:10]["personaname"]

In [9]:
#for game in games["game_info"][0]:
#    print game["playtime_forever"], game["name"]

# Unpack Columns Containing a Single Dictionary

In [242]:
def unpack(df, column, fillna=None):
    new_df = None
    if fillna is None:
        new_df = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del new_df[column]
    else:
        new_df = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del new_df[column]
    return new_df

#unpack(tf, 'nested', 0)

In [243]:
df = unpack(df, "user_info", fillna=None)

In [244]:
df = unpack(df, "ban_status", fillna=None)

In [248]:
df.head(1)

,friends,game_info,profile_summary,steamid,avatarfull,commentpermission,communityvisibilitystate,gameextrainfo,gameid,gameserverip,...,primaryclanid,profilestate,realname,timecreated,CommunityBanned,DaysSinceLastBan,EconomyBan,NumberOfGameBans,NumberOfVACBans,VACBanned
0,"[{u'steamid': u'76561197962935145', u'friend_s...","[{u'steamid': u'76561197981918203', u'playtime...",No information given.,76561197981918203,https://steamcdn-a.akamaihd.net/steamcommunity...,NaN,3,NaN,NaN,NaN,...,103582791429521408,1.0,NaN,1.144367e+09,False,0,none,0,0,False


In [20]:
#for game in games["game_info"][10]:
#    print game["name"], game["playtime_forever"]

In [72]:
few_games = df[["game_info", "steamid"]]

In [94]:
IdSeries = few_games["steamid"]

In [138]:
pub_games = games.copy()
pub_games = games[games["game_info"]!="private"].copy()

In [61]:
pub_games = pub_games.set_index("steamid")

In [96]:
pub_games.iloc[2].values

array([ [{u'playtime_forever': 84, u'playtime_2weeks': 81, u'name': u'Path of Exile', u'appid': 238960}, {u'playtime_forever': 9, u'playtime_2weeks': 9, u'name': u'Unturned', u'appid': 304930}]], dtype=object)

In [26]:
import numpy as np

In [142]:
pub_games.head(10)

,steamid,game_info
0,76561197981918203,"[{u'uid': u'76561197981918203', u'playtime_for..."
1,76561197993408842,"[{u'uid': u'76561197993408842', u'playtime_for..."
2,76561198042305043,"[{u'uid': u'76561198042305043', u'playtime_for..."
4,76561197980092471,"[{u'uid': u'76561197980092471', u'playtime_for..."
5,76561198202663196,"[{u'uid': u'76561198202663196', u'playtime_for..."
6,76561198129757415,"[{u'uid': u'76561198129757415', u'playtime_for..."
7,76561198050035478,"[{u'uid': u'76561198050035478', u'playtime_for..."
8,76561197976676146,"[{u'uid': u'76561197976676146', u'playtime_for..."
9,76561197995295208,"[{u'uid': u'76561197995295208', u'playtime_for..."
10,76561198068277537,"[{u'uid': u'76561198068277537', u'playtime_for..."


In [140]:
pub_games = pub_games[pd.notnull(pub_games["steamid"])]

In [40]:
game_names = list(map(lambda x: x["name"], pub_games["game_info"][0]))

In [204]:
import os

In [205]:
os.environ["MY_STEAM"]

'76561197967398882'

In [32]:
pub_games[np.array(map(lambda x: len(x), pub_games["game_info"])) == 1]

,steamid,game_info
55,76561198047666691,"[{u'playtime_forever': 2241, u'playtime_2weeks..."
110,76561197998721109,"[{u'playtime_forever': 244, u'playtime_2weeks'..."
137,76561198010718118,"[{u'playtime_forever': 74, u'playtime_2weeks':..."
158,76561198200656075,"[{u'playtime_forever': 380, u'playtime_2weeks'..."
183,76561198001352634,"[{u'playtime_forever': 440, u'playtime_2weeks'..."
216,76561198082073715,"[{u'playtime_forever': 9317, u'playtime_2weeks..."
221,76561198178311941,"[{u'playtime_forever': 0, u'playtime_2weeks': ..."
223,76561198290216463,"[{u'playtime_forever': 2, u'playtime_2weeks': ..."
270,76561198134958228,"[{u'playtime_forever': 49, u'playtime_2weeks':..."
370,76561198042114259,"[{u'playtime_forever': 109, u'playtime_2weeks'..."


# Pivot Table

In [344]:
def sparse_games(df, vals="playtime_forever"):
    """
    Assumes a df with public non-null values in the form:
    |   |steamid          |game_info        |
    +---+-----------------+-----------------+
    |0  |76561197967398882|[{u'playt...},{}]|
    
    Returns a pivoted data fram where each row is
    'steamid', the columns are game names, and the 
    values are 'playtime_forever' (default) or 
    'playtime_2weeks'
    """
    df = df.set_index("steamid")
    for i in xrange(len(df)):
        for game in df.iloc[i]["game_info"]:
            game["uid"] = df.iloc[i].name
    
    sparsegames = pd.DataFrame()
    for user in df["game_info"]:
        gamedf = pd.DataFrame(user)
        sparsegames = sparsegames.append(gamedf)
    
    
    pivoted = pd.pivot_table(SparseGames, index="uid", columns=["appid"], values=[vals], fill_value=0)
    return pivoted.playtime_forever 
    
    

In [345]:
pivoted = sparse_games(pub_games)

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 76561197976676146 to 76561198202663196
Columns: 518 entries, 10.0 to 433340.0
dtypes: int64(518)
memory usage: 40.5+ KB


In [332]:
game_times.mean()

1977.449806949807

In [346]:
game_times = np.array([])
for game in pivoted:
    #print Pivoted.playtime_2weeks[game].sum()
    game_times = np.append(game_times, pivoted[game].sum())
print game_times.mean()

1977.44980695


In [280]:
Pivoted.playtime_2weeks[730].sum()/60.

1031.1333333333334

In [308]:
game_times.mean()

15227.731133113311

In [311]:
Pivoted.playtime_forever.columns[:10]

Int64Index([10, 20, 30, 40, 50, 60, 70, 80, 92, 100], dtype='int64', name=u'appid')

# Drop columns with with total playtime below average

In [348]:
for col in pivoted.columns:
    if pivoted[col].sum() <= 1977:
        pivoted.drop(col, axis=1, inplace=True)

In [337]:
#Pivoted.playtime_forever.drop(10, axis=1)
pivoted.head()[730]

uid
76561197976676146    1588
76561197980092471       0
76561197981918203     234
76561197993408842       0
76561197995295208       0
Name: 730.0, dtype: int64

In [125]:
sample.iloc[2].name

u'76561198042305043'

In [126]:
for i in xrange(len(sample)):
    #print i
    for g in sample.iloc[i]["game_info"]:
        #print len(g)
        #print g["name"], g["playtime_forever"], pub_games["steamid"][i]
        g["uid"] = sample.iloc[i].name
        #print pub_games["steamid"][i]
        #del g["appid"]
    #print game[0]

In [43]:
#few_games["game_info"][0]

[{u'name': u'Half-Life 2',
  u'playtime_2weeks': 0,
  u'playtime_forever': 63,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Counter-Strike: Source',
  u'playtime_2weeks': 0,
  u'playtime_forever': 72,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Half-Life: Source',
  u'playtime_2weeks': 0,
  u'playtime_forever': 0,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Half-Life 2: Deathmatch',
  u'playtime_2weeks': 0,
  u'playtime_forever': 0,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Half-Life 2: Lost Coast',
  u'playtime_2weeks': 0,
  u'playtime_forever': 25,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Half-Life Deathmatch: Source',
  u'playtime_2weeks': 0,
  u'playtime_forever': 0,
  'steamid': u'76561197981918203',
  'uid': u'76561197981918203'},
 {u'name': u'Half-Life 2: Episode One',
  u'playtime_2weeks': 0,
  u'

In [135]:
GameSeries = few_games["game_info"][1]
GameSeries2 = pd.DataFrame(few_games["game_info"][2])

In [115]:
SparseGames = pd.DataFrame(GameSeries)

In [177]:
SparseGames.head()

,appid,name,playtime_2weeks,playtime_forever,uid
0,NaN,Half-Life 2,0.0,2893.0,76561197971340950
1,NaN,Counter-Strike: Source,0.0,1711.0,76561197971340950
2,NaN,Half-Life 2: Deathmatch,0.0,2090.0,76561197971340950
3,NaN,Half-Life 2: Lost Coast,0.0,52.0,76561197971340950
4,NaN,Garry's Mod,43.0,26780.0,76561197971340950


In [125]:
for u in few_games["game_info"]:
    user_df = pd.DataFrame(u)
    SparseGames.append(user_df)

In [128]:
SparseGames = pd.DataFrame(columns=["name", "playtime_2weeks", "playtime_forever", "uid"])

In [129]:
#SparseGames = pd.D
for u in sample["game_info"]:
    Gamedf = pd.DataFrame(u)
    SparseGames = SparseGames.append(Gamedf)

,appid,name,playtime_2weeks,playtime_forever,uid
0,220.0,Half-Life 2,0.0,63.0,76561197981918203
1,240.0,Counter-Strike: Source,0.0,72.0,76561197981918203
2,280.0,Half-Life: Source,0.0,0.0,76561197981918203
3,320.0,Half-Life 2: Deathmatch,0.0,0.0,76561197981918203
4,340.0,Half-Life 2: Lost Coast,0.0,25.0,76561197981918203


In [169]:
#SparseGames = pd.DataFrame(columns=["name", "playtime_2weeks", "playtime_forever", "uid"])

In [52]:
del SparseGames["appid"]
del SparseGames["steamid"]

In [53]:
SparseGames.head()

,name,playtime_2weeks,playtime_forever,uid
0,Half-Life 2,0.0,63.0,76561197981918203
1,Counter-Strike: Source,0.0,72.0,76561197981918203
2,Half-Life: Source,0.0,0.0,76561197981918203
3,Half-Life 2: Deathmatch,0.0,0.0,76561197981918203
4,Half-Life 2: Lost Coast,0.0,25.0,76561197981918203


In [132]:
# df.pivot(index='date', columns='variable', values='value')
Pivoted = pd.pivot_table(SparseGames, index=uid, columns=["name"], values=["playtime_forever"], fill_value=0)

In [135]:
Pivoted

playtime_forever                \
name              A Virus Named TOM Ace of Spades   
uid                                                 
76561197976676146                 0             0   
76561197980092471                 0             0   
76561197981918203                 0             0   
76561197993408842                 0           104   
76561197995295208                 0             0   
76561198042305043                 0             0   
76561198050035478                 0             0   
76561198068277537                 0             0   
76561198129757415                 0             0   
76561198202663196                 0             0   

                                                 \
name              Age of Empires II: HD Edition   
uid                                               
76561197976676146                            33   
76561197980092471                             0   
76561197981918203                           256   
76561197993408842                             0   
76561197995295208                             0   
76561198042305043                             0   
76561198050035478                             0   
76561198068277537                             0   
76561198129757415                             0   
76561198202663196                             0   

                                                                      \
name              Age of Empires® III: Complete Collection Alan Wake   
uid                                                                    
76561197976676146                                        8         0   
76561197980092471                                        0        27   
76561197981918203                                        0         0   
76561197993408842                                        0         0   
76561197995295208                                        0         0   
76561198042305043                                        0         0   
76561198050035478                                        0         0   
76561198068277537                                        0         0   
76561198129757415                                        0         0   
76561198202663196                                        0         0   

                                                                               \
name              Alan Wake's American Nightmare Alien Swarm Alien: Isolation   
uid                                                                             
76561197976676146                              0        1810                0   
76561197980092471                              0         421                0   
76561197981918203                              0           0              384   
76561197993408842                              0           0                0   
76561197995295208                              0           0                0   
76561198042305043                              0           0                0   
76561198050035478                              0           0                0   
76561198068277537                              0           0                0   
76561198129757415                              0           0                0   
76561198202663196                              0           0                0   

                                                                       ...     \
name              Alpha Protocol Amazing Adventures Around the World   ...      
uid                                                                    ...      
76561197976676146              0                                   0   ...      
76561197980092471            141                                   0   ...      
76561197981918203              0                                   0   ...      
76561197993408842              0                                   0   ...      
76561197995295208              0                                   0   ...      
76561198042305043         